In [143]:
%matplotlib inline
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_distribution, array_to_latex, plot_bloch_multivector
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector
from qiskit.circuit import Parameter
from qiskit_aer import AerSimulator
from qiskit.quantum_info.operators import Operator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

![diagram](./assets/buckshot-roulette-diagram.png)

In [200]:
player_strategy = lambda bullets, fakes: "himself" if(bullets == fakes or fakes > bullets) else "shoot"
dealer_strategy = lambda bullets, fakes:  "himself" if np.random.randint(2) else "shoot"

In [206]:
def gun(bullets, fakes) -> bool:
    v = bullets + fakes
    r = np.random.randint(1, v+1)
    return r <= bullets
    

def play(strategies:dict, players:list):
    total_bullets = 3
    total_fakes = 2

    lives = {
        "player":0,
        "dealer":0
    }

    actual_player = players[0]
    oponent = players[1]
    
    while(total_bullets):
        player = strategies[actual_player]
        action = player(total_bullets, total_fakes)
        shot = gun(total_bullets, total_fakes)
        
        print(f"playing: {actual_player}")
        print(f"oponent: {oponent}")
        print(f"action: {action}")
        print(f"shot: {shot}")
        

        if(action == "shoot" and shot):
            lives[oponent] -=1
            total_bullets -=1
            actual_player = players[0]
            oponent = players[1]

        elif(action == "shoot" and not shot):
            total_fakes -=1
            actual_player,oponent = oponent,actual_player

        elif(action == "himself" and shot):
            lives[actual_player] -=1
            total_bullets -=1
            actual_player = players[0]
            oponent = players[1]

        else:
            total_fakes -=1            

        print(lives, end="\n\n")


In [210]:
play({"player":player_strategy, "dealer":dealer_strategy}, ("player", "dealer"))

playing: player
oponent: dealer
action: shoot
shot: False
{'player': 0, 'dealer': 0}

playing: dealer
oponent: player
action: himself
shot: True
{'player': 0, 'dealer': -1}

playing: player
oponent: dealer
action: shoot
shot: True
{'player': 0, 'dealer': -2}

playing: player
oponent: dealer
action: himself
shot: True
{'player': -1, 'dealer': -2}

